In [1]:
import pysmile
import pysmile_license
import numpy as np
import pandas as pd

from network_functions import calculate_network_utilities, new_screening_strategy, old_screening_strategy, create_folders_logger
from simulations import plot_classification_results
from plots import plot_estimations_w_error_bars, plot_screening_counts

from preprocessing import preprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
import matplotlib.pyplot as plt

import yaml
with open('config.yaml', 'r') as file:
    cfg = yaml.safe_load(file)

import argparse
import pdb
import matplotlib
matplotlib.use('Agg')

import logging
import datetime 
import os

In [2]:
# from main import main
from use_case_new_strategy import use_case_new_strategy
from full_example import full_example
from update import update_influence_diagram
from network_functions import create_folders_logger
from sens_analysis_elicitation import sens_analysis_elicitation
from sens_analysis_PE_method import sens_analysis_PE_method

### Preferences model

In [3]:
cfg["patient_chars"]

{'Age': 'age_4_adult',
 'Sex': 'M',
 'SD': 'SD_2_normal',
 'PA': 'PA_2',
 'Smoking': 'sm_1_not_smoker',
 'BMI': 'bmi_2_normal',
 'Alcohol': 'low'}

## Use cases

### Individual cases

**Benchmark patient**

In [4]:
label = 'notebook_benchmark_patient'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2024-11-21 15:52:12,576 - INFO - Model type: linear
2024-11-21 15:52:12,577 - INFO - Reading network...
2024-11-21 15:52:12,600 - INFO - Calculating information values...
2024-11-21 15:53:22,348 - INFO - Eliciting value of comfort...
2024-11-21 15:53:22,349 - INFO - No elicitation of lambda values, taking default values...
2024-11-21 15:53:22,350 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-21 15:53:22,351 - INFO - Saving network...
2024-11-21 15:53:22,397 - INFO - Plotting info functions...
2024-11-21 15:53:56,490 - INFO - Calculating final utilities...
2024-11-21 15:53:56,496 - INFO - Best info: 15.299634978175163
2024-11-21 15:53:56,497 - INFO - Worst info: 4.450147717014403e-308
2024-11-21 15:53:56,498 - INFO - PE info: 4
2024-11-21 15:53:56,498 - INFO - v_best: 104.06788025218766
2024-11-21 15:53:56,498 - INFO - v_worst: -3.91023528629925

**Benchmark patient risk seeking**

In [5]:
label = 'notebook_benchmark_risk_seeking'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    change_risk_param=True,
    rho_param= 0.005
)

2024-11-21 15:53:56,677 - INFO - Model type: linear
2024-11-21 15:53:56,677 - INFO - Model type: linear
2024-11-21 15:53:56,678 - INFO - Reading network...
2024-11-21 15:53:56,678 - INFO - Reading network...
2024-11-21 15:53:56,696 - INFO - Calculating information values...
2024-11-21 15:53:56,696 - INFO - Calculating information values...
2024-11-21 15:54:47,911 - INFO - Eliciting value of comfort...
2024-11-21 15:54:47,911 - INFO - Eliciting value of comfort...
2024-11-21 15:54:47,913 - INFO - No elicitation of lambda values, taking default values...
2024-11-21 15:54:47,913 - INFO - No elicitation of lambda values, taking default values...
2024-11-21 15:54:47,915 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-21 15:54:47,915 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.8019

**Benchmark patient added evidence**

In [6]:
label = 'notebook_benchmark_addded_ev'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = cfg["patient_chars"].copy()
patient_chars["Diabetes"] = True
patient_chars["Hypertension"] = True

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
)

2024-11-21 15:55:21,915 - INFO - Model type: linear
2024-11-21 15:55:21,916 - INFO - Reading network...
2024-11-21 15:55:21,934 - INFO - Calculating information values...
2024-11-21 15:56:14,156 - INFO - Eliciting value of comfort...
2024-11-21 15:56:14,157 - INFO - No elicitation of lambda values, taking default values...
2024-11-21 15:56:14,159 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-21 15:56:14,159 - INFO - Saving network...
2024-11-21 15:56:14,216 - INFO - Plotting info functions...
2024-11-21 15:56:48,828 - INFO - Calculating final utilities...
2024-11-21 15:56:48,835 - INFO - Best info: 15.299634978175163
2024-11-21 15:56:48,835 - INFO - Worst info: 4.450147717014403e-308
2024-11-21 15:56:48,836 - INFO - PE info: 4
2024-11-21 15:56:48,836 - INFO - v_best: 104.06788025218766
2024-11-21 15:56:48,837 - INFO - v_worst: -3.91023528629925

**Exogenous variable p = 0.02**

In [7]:
label = 'notebook_exogenous_var_002'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

exog_char ={"Age": "age_2_young", 
    "Sex": "M",
    "SES": "ses_0",
    "SD": "SD_1_short",
    "PA": "PA_1",
    "Depression": False,
    "Smoking": "sm_1_not_smoker",
    "Anxiety": False,
    "BMI": "bmi_1_underweight",
    "Alcohol": "high",
    "Diabetes": False,
    "Hypertension": False,
    "Hyperchol_": False,
    }


update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = exog_char,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    exogenous_var_prob= 0.02
)

2024-11-21 15:56:48,954 - INFO - Model type: linear
2024-11-21 15:56:48,954 - INFO - Reading network...
2024-11-21 15:56:48,972 - INFO - Calculating information values...
2024-11-21 15:57:41,115 - INFO - Eliciting value of comfort...
2024-11-21 15:57:41,117 - INFO - No elicitation of lambda values, taking default values...
2024-11-21 15:57:41,118 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-21 15:57:41,118 - INFO - Saving network...
2024-11-21 15:57:41,175 - INFO - Plotting info functions...
2024-11-21 15:58:15,337 - INFO - Calculating final utilities...
2024-11-21 15:58:15,344 - INFO - Best info: 15.299634978175163
2024-11-21 15:58:15,345 - INFO - Worst info: 4.450147717014403e-308
2024-11-21 15:58:15,345 - INFO - PE info: 4
2024-11-21 15:58:15,346 - INFO - v_best: 104.06788025218766
2024-11-21 15:58:15,346 - INFO - v_worst: -3.91023528629925

**Exogenous variable p = 0.1**

In [8]:
label = 'notebook_exogenous_var_01'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

exog_char ={"Age": "age_2_young", 
    "Sex": "M",
    "SES": "ses_0",
    "SD": "SD_1_short",
    "PA": "PA_1",
    "Depression": False,
    "Smoking": "sm_1_not_smoker",
    "Anxiety": False,
    "BMI": "bmi_1_underweight",
    "Alcohol": "high",
    "Diabetes": False,
    "Hypertension": False,
    "Hyperchol_": False,
    }

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = exog_char,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label,
    exogenous_var_prob= 0.1
)

2024-11-21 15:58:15,459 - INFO - Model type: linear
2024-11-21 15:58:15,460 - INFO - Reading network...
2024-11-21 15:58:15,477 - INFO - Calculating information values...
2024-11-21 15:59:07,883 - INFO - Eliciting value of comfort...
2024-11-21 15:59:07,884 - INFO - No elicitation of lambda values, taking default values...
2024-11-21 15:59:07,886 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-21 15:59:07,886 - INFO - Saving network...
2024-11-21 15:59:07,942 - INFO - Plotting info functions...
2024-11-21 15:59:42,111 - INFO - Calculating final utilities...
2024-11-21 15:59:42,118 - INFO - Best info: 15.299634978175163
2024-11-21 15:59:42,118 - INFO - Worst info: 4.450147717014403e-308
2024-11-21 15:59:42,119 - INFO - PE info: 4
2024-11-21 15:59:42,119 - INFO - v_best: 104.06788025218766
2024-11-21 15:59:42,120 - INFO - v_worst: -3.91023528629925

### Assessing the French strategy

**Patient 1**

In [9]:
label = 'notebook_4_2_patient_1'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = {
    "Age": "age_5_old_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_2",
    "Smoking": "sm_1_not_smoker",
    "BMI": "bmi_2_normal",
    "Alcohol": "low",
    "Diabetes": False,
    "Hypertension": False,
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2024-11-21 15:59:42,289 - INFO - Model type: linear
2024-11-21 15:59:42,290 - INFO - Reading network...
2024-11-21 15:59:42,308 - INFO - Calculating information values...
2024-11-21 16:00:34,581 - INFO - Eliciting value of comfort...
2024-11-21 16:00:34,582 - INFO - No elicitation of lambda values, taking default values...
2024-11-21 16:00:34,583 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-21 16:00:34,584 - INFO - Saving network...
2024-11-21 16:00:34,639 - INFO - Plotting info functions...
2024-11-21 16:01:09,207 - INFO - Calculating final utilities...
2024-11-21 16:01:09,214 - INFO - Best info: 15.299634978175163
2024-11-21 16:01:09,215 - INFO - Worst info: 4.450147717014403e-308
2024-11-21 16:01:09,215 - INFO - PE info: 4
2024-11-21 16:01:09,216 - INFO - v_best: 104.06788025218766
2024-11-21 16:01:09,216 - INFO - v_worst: -3.91023528629925

**Patient 2**

In [10]:
label = 'notebook_4_2_patient_2'
logger, log_dir = create_folders_logger(label = label, output_dir = 'notebook/' + label, date = False, time = False)

patient_chars = {
    "Age": "age_4_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_2",
    "Smoking": "sm_1_not_smoker",
    "BMI": "bmi_2_normal",
    "Alcohol": "low",
    "Diabetes": True,
    "Hypertension": True,
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2024-11-21 16:01:09,333 - INFO - Model type: linear
2024-11-21 16:01:09,334 - INFO - Reading network...
2024-11-21 16:01:09,352 - INFO - Calculating information values...
2024-11-21 16:02:01,126 - INFO - Eliciting value of comfort...
2024-11-21 16:02:01,128 - INFO - No elicitation of lambda values, taking default values...
2024-11-21 16:02:01,129 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-21 16:02:01,130 - INFO - Saving network...
2024-11-21 16:02:01,186 - INFO - Plotting info functions...
2024-11-21 16:02:35,415 - INFO - Calculating final utilities...
2024-11-21 16:02:35,422 - INFO - Best info: 15.299634978175163
2024-11-21 16:02:35,422 - INFO - Worst info: 4.450147717014403e-308
2024-11-21 16:02:35,423 - INFO - PE info: 4
2024-11-21 16:02:35,423 - INFO - v_best: 104.06788025218766
2024-11-21 16:02:35,424 - INFO - v_worst: -3.91023528629925

**Patient 3**

In [11]:
label = 'notebook_4_2_patient_3'
logger, log_dir = create_folders_logger(label = label)

patient_chars = {
    "Age": "age_4_adult", 
    "Sex": "M",
    "SD": "SD_2_normal",
    "PA": "PA_1",
    "Smoking": "sm_3_ex_smoker",
    "BMI": "bmi_3_overweight",
    "Alcohol": "high",
}

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = patient_chars,
    new_test = cfg["new_test"],
    logger = logger,
    output_dir = 'notebook/' + label
)

2024-11-21 16:02:35,532 - INFO - Model type: linear
2024-11-21 16:02:35,533 - INFO - Reading network...
2024-11-21 16:02:35,550 - INFO - Calculating information values...
2024-11-21 16:03:27,585 - INFO - Eliciting value of comfort...
2024-11-21 16:03:27,586 - INFO - No elicitation of lambda values, taking default values...
2024-11-21 16:03:27,588 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-21 16:03:27,588 - INFO - Saving network...
2024-11-21 16:03:27,632 - INFO - Plotting info functions...
2024-11-21 16:04:01,608 - INFO - Calculating final utilities...
2024-11-21 16:04:01,615 - INFO - Best info: 15.299634978175163
2024-11-21 16:04:01,616 - INFO - Worst info: 4.450147717014403e-308
2024-11-21 16:04:01,617 - INFO - PE info: 4
2024-11-21 16:04:01,617 - INFO - v_best: 104.06788025218766
2024-11-21 16:04:01,618 - INFO - v_worst: -3.91023528629925

### Designing a national personalised strategy

In [12]:
label = 'notebook_personalised_screening_str'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

use_case_new_strategy(
    logger = logger,
    single_run=True,
    output_dir = 'notebook/' + label,
    log_dir = log_dir,
    run_label = ''
)

2024-11-21 16:04:01,733 - INFO - Configuration variables of interest:
2024-11-21 16:04:01,734 - INFO - Single run: True
2024-11-21 16:04:01,734 - INFO - Use all variables: True
2024-11-21 16:04:01,735 - INFO - Use case with new test: False
2024-11-21 16:04:01,735 - INFO - PE method: {'PE_cost': 5, 'PE_info': 4, 'PE_prob': 0.7}
2024-11-21 16:04:01,736 - INFO - Reading the network file...
2024-11-21 16:04:01,753 - INFO - Located at: outputs/linear_rel_point_cond_mut_info_elicitFalse_newtestFalse/decision_models/DM_screening_rel_point_cond_mut_info_linear.xdsl
2024-11-21 16:04:01,754 - INFO - Comfort values: 1 - 4.011942483017116, 2 - 4.168480067521723, 3 - 6.801984517973132, 4 - 7.0
2024-11-21 16:04:02,422 - INFO - All variables are kept in the dataframe for calculation of utilities.
2024-11-21 16:04:02,422 - INFO - Operational limits for the screening strategies: {'No_scr_no_col': inf, 'No_scr_col': 3000, 'gFOBT': 30000, 'FIT': 42000, 'Blood_based': 7000, 'Stool_DNA': 6000, 'CTC': 2000,

### Benchmarking of new screening devices

In [13]:
cfg["patient_chars"]

{'Age': 'age_4_adult',
 'Sex': 'M',
 'SD': 'SD_2_normal',
 'PA': 'PA_2',
 'Smoking': 'sm_1_not_smoker',
 'BMI': 'bmi_2_normal',
 'Alcohol': 'low'}

In [14]:
label = 'notebook_benchmark_new_screening'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

update_influence_diagram(
    model_type = cfg["model_type"],
    value_function = cfg["value_function"],
    elicit = cfg["elicit"],
    noise = cfg["noise"],
    calculate_info_values= cfg["calculate_info_values"],
    ref_patient_chars = cfg["patient_chars"],
    new_test = True,
    logger = logger,
    output_dir = 'notebook/' + label
)

2024-11-21 16:05:53,829 - INFO - Model type: linear
2024-11-21 16:05:53,830 - INFO - Reading network...
2024-11-21 16:05:53,849 - INFO - Calculating information values...
2024-11-21 16:06:46,119 - INFO - Eliciting value of comfort...
2024-11-21 16:06:46,120 - INFO - No elicitation of lambda values, taking default values...
2024-11-21 16:06:46,121 - INFO - Lambda values: [7.         4.01194248 6.80198452 4.01194248 6.80198452 4.01194248
 6.80198452 4.01194248 6.80198452 4.01194248 4.16848007 4.01194248
 4.16848007 4.01194248]
2024-11-21 16:06:46,122 - INFO - Adding new test values...
2024-11-21 16:07:53,059 - INFO - Saving network...
2024-11-21 16:07:53,110 - INFO - Plotting info functions...
2024-11-21 16:09:13,827 - INFO - Calculating final utilities...
2024-11-21 16:09:13,836 - INFO - Best info: 15.299634978175163
2024-11-21 16:09:13,836 - INFO - Worst info: 4.450147717014403e-308
2024-11-21 16:09:13,837 - INFO - PE info: 4
2024-11-21 16:09:13,837 - INFO - v_best: 104.06788025218766


In [15]:
cfg["lambda_list"]

[4.011942483017116, 4.168480067521723, 6.801984517973132, 7.0]

In [16]:
label = 'notebook_new_test_screening_strategy'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

use_case_new_strategy(
    use_case_new_test = True,
    single_run=True,
    output_dir = 'notebook/' + label,
    log_dir = log_dir,
    run_label = '',
)

2024-11-21 16:09:14,004 - INFO - Configuration variables of interest:
2024-11-21 16:09:14,004 - INFO - Configuration variables of interest:
2024-11-21 16:09:14,005 - INFO - Single run: True
2024-11-21 16:09:14,005 - INFO - Single run: True
2024-11-21 16:09:14,006 - INFO - Use all variables: True
2024-11-21 16:09:14,006 - INFO - Use all variables: True
2024-11-21 16:09:14,007 - INFO - Use case with new test: True
2024-11-21 16:09:14,007 - INFO - Use case with new test: True
2024-11-21 16:09:14,008 - INFO - PE method: {'PE_cost': 5, 'PE_info': 4, 'PE_prob': 0.7}
2024-11-21 16:09:14,008 - INFO - PE method: {'PE_cost': 5, 'PE_info': 4, 'PE_prob': 0.7}
2024-11-21 16:09:14,009 - INFO - Reading the network file...
2024-11-21 16:09:14,009 - INFO - Reading the network file...
2024-11-21 16:09:14,030 - INFO - Located at: outputs/linear_rel_point_cond_mut_info_elicitFalse_newtestTrue/decision_models/DM_screening_rel_point_cond_mut_info_linear_new_test.xdsl
2024-11-21 16:09:14,030 - INFO - Located

## Sensitivity analysis

### Sensitivity analysis for the PE

In [17]:
label = 'notebook_sens_analysis_PE_method'
logger, log_dir = create_folders_logger(label=label, output_dir = 'notebook/' + label, date = False, time = False)

sens_analysis_PE_method(
    label = label,
    output_dir = 'notebook/' + label,
    log_dir = log_dir,  
)

2024-11-21 16:11:27,357 - INFO - PE_info_array: [3.8 4.  4.2]
2024-11-21 16:11:27,357 - INFO - PE_info_array: [3.8 4.  4.2]
2024-11-21 16:11:27,358 - INFO - PE_cost_array: [ 5 10 50]


2024-11-21 16:11:27,358 - INFO - PE_cost_array: [ 5 10 50]
2024-11-21 16:11:31,642 - INFO - Params: [1.010540555812223, 0.8618081053948691, 0.040715714514626765]
2024-11-21 16:11:31,642 - INFO - Params: [1.010540555812223, 0.8618081053948691, 0.040715714514626765]
2024-11-21 16:12:37,441 - INFO - PE_info: 3.8, PE_cost: 5, DONE!
2024-11-21 16:12:37,441 - INFO - PE_info: 3.8, PE_cost: 5, DONE!
2024-11-21 16:12:38,201 - INFO - Params: [1.0170169518491206, 0.8677488181646118, 0.04059263956915226]
2024-11-21 16:12:38,201 - INFO - Params: [1.0170169518491206, 0.8677488181646118, 0.04059263956915226]
2024-11-21 16:13:42,116 - INFO - PE_info: 3.8, PE_cost: 10, DONE!
2024-11-21 16:13:42,116 - INFO - PE_info: 3.8, PE_cost: 10, DONE!
2024-11-21 16:13:42,888 - INFO - Params: [1.0151957192545649, 0.8624584320399733, 0.04169819327814257]
2024-11-21 16:13:42,888 - INFO - Params: [1.0151957192545649, 0.8624584320399733, 0.04169819327814257]
2024-11-21 16:14:47,718 - INFO - PE_info: 3.8, PE_cost: 50, D

### Sensitivity analysis for comfort parameters

In [18]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

sens_analysis_elicitation(
    label = label,
    output_dir = 'notebook/' + label
)

2024-11-21 16:21:17,144 - INFO - Configuration variables of interest:
2024-11-21 16:21:17,144 - INFO - Configuration variables of interest:
2024-11-21 16:21:17,144 - INFO - Configuration variables of interest:
2024-11-21 16:21:17,144 - INFO - Configuration variables of interest:
2024-11-21 16:21:17,146 - INFO - Single run: True
2024-11-21 16:21:17,146 - INFO - Single run: True
2024-11-21 16:21:17,146 - INFO - Single run: True
2024-11-21 16:21:17,146 - INFO - Single run: True
2024-11-21 16:21:17,147 - INFO - Use all variables: True
2024-11-21 16:21:17,147 - INFO - Use all variables: True
2024-11-21 16:21:17,147 - INFO - Use all variables: True
2024-11-21 16:21:17,147 - INFO - Use all variables: True
2024-11-21 16:21:17,149 - INFO - Use case with new test: False
2024-11-21 16:21:17,149 - INFO - Use case with new test: False
2024-11-21 16:21:17,149 - INFO - Use case with new test: False
2024-11-21 16:21:17,149 - INFO - Use case with new test: False
2024-11-21 16:21:17,151 - INFO - PE meth

Counts: best_option
No_scr_no_col    287920
No_scr_col            0
gFOBT                 0
FIT                4131
Blood_based           0
Stool_DNA         47656
CTC                   0
Colon_capsule         0
Name: count, dtype: int64


In [19]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

lambda_list = cfg["lambda_list"].copy()
lambda_list[2] = 6.3

sens_analysis_elicitation(
    lambda_list = lambda_list,
    label = label,
    output_dir = 'notebook/' + label
)

2024-11-21 16:23:52,384 - INFO - Configuration variables of interest:
2024-11-21 16:23:52,384 - INFO - Configuration variables of interest:
2024-11-21 16:23:52,385 - INFO - Single run: True
2024-11-21 16:23:52,385 - INFO - Single run: True
2024-11-21 16:23:52,387 - INFO - Use all variables: True
2024-11-21 16:23:52,387 - INFO - Use all variables: True
2024-11-21 16:23:52,388 - INFO - Use case with new test: False
2024-11-21 16:23:52,388 - INFO - Use case with new test: False
2024-11-21 16:23:52,389 - INFO - PE method: {'PE_cost': 5, 'PE_info': 4, 'PE_prob': 0.7}
2024-11-21 16:23:52,389 - INFO - PE method: {'PE_cost': 5, 'PE_info': 4, 'PE_prob': 0.7}
2024-11-21 16:23:52,390 - INFO - Change lambdas? True
2024-11-21 16:23:52,390 - INFO - Change lambdas? True
2024-11-21 16:23:52,391 - INFO - Read lambda list from config? True
2024-11-21 16:23:52,391 - INFO - Read lambda list from config? True
2024-11-21 16:23:52,392 - INFO - Lambdas: [4.011942483017116, 4.168480067521723, 6.3, 7.0]
2024-11

Counts: best_option
No_scr_no_col    313631
No_scr_col            0
gFOBT                 0
FIT                   0
Blood_based           0
Stool_DNA         26076
CTC                   0
Colon_capsule         0
Name: count, dtype: int64


In [20]:
label = 'notebook_sens_analysis_comf_param'
logger, log_dir = create_folders_logger(label = label, output_dir= 'notebook/' + label, date = False, time = False)

lambda_list = cfg["lambda_list"]
lambda_list[0] = 4.8
lambda_list[1] = 5

sens_analysis_elicitation(
    lambda_list = lambda_list,
    label = label,
    output_dir = 'notebook/' + label
)

2024-11-21 16:26:25,592 - INFO - Configuration variables of interest:
2024-11-21 16:26:25,592 - INFO - Configuration variables of interest:
2024-11-21 16:26:25,593 - INFO - Single run: True
2024-11-21 16:26:25,593 - INFO - Single run: True
2024-11-21 16:26:25,594 - INFO - Use all variables: True
2024-11-21 16:26:25,594 - INFO - Use all variables: True
2024-11-21 16:26:25,595 - INFO - Use case with new test: False
2024-11-21 16:26:25,595 - INFO - Use case with new test: False
2024-11-21 16:26:25,596 - INFO - PE method: {'PE_cost': 5, 'PE_info': 4, 'PE_prob': 0.7}
2024-11-21 16:26:25,596 - INFO - PE method: {'PE_cost': 5, 'PE_info': 4, 'PE_prob': 0.7}
2024-11-21 16:26:25,597 - INFO - Change lambdas? True
2024-11-21 16:26:25,597 - INFO - Change lambdas? True
2024-11-21 16:26:25,598 - INFO - Read lambda list from config? True
2024-11-21 16:26:25,598 - INFO - Read lambda list from config? True
2024-11-21 16:26:25,599 - INFO - Lambdas: [4.8, 5, 6.801984517973132, 7.0]
2024-11-21 16:26:25,599

Counts: best_option
No_scr_no_col    179713
No_scr_col            0
gFOBT                 0
FIT                   0
Blood_based           0
Stool_DNA        159994
CTC                   0
Colon_capsule         0
Name: count, dtype: int64
